In [1]:
using Requests: get, json

In [2]:
function reverse_geocoding(latitude, longitude; api = :google)
    APIs = Dict(
        :google => "https://maps.googleapis.com/maps/api/geocode/json?latlng=$latitude,$longitude",
        :openstreetmap => "http://nominatim.openstreetmap.org/reverse?format=json&lat=$latitude&lon=$longitude"
    )
    response = get(APIs[api])
    return response.status == 200 ? json(response) : response
end

reverse_geocoding (generic function with 1 method)

In [3]:
data = readdlm("reverse_geocoding.csv", ',')

100000×5 Array{Float64,2}:
      1.0  19.1599   -99.1423  19.1906   -99.1398
      2.0  19.1361   -99.2596  19.1547   -99.0275
      3.0  20.7293  -100.49    19.3494  -101.225 
      4.0  17.5787  -100.901   20.1571  -100.12  
      5.0  20.1634   -97.2348  19.4832   -99.1224
      6.0  20.9918   -99.7457  20.2681   -99.9012
      7.0  16.9768   -98.0301  19.4797   -99.1828
      8.0  18.6683   -98.9942  19.0884   -99.0114
      9.0  19.4667   -99.0565  19.1166   -99.1408
     10.0  17.3611   -94.5553  16.4153   -96.6446
     11.0  17.6689   -94.1377  19.2037   -99.31  
     12.0  19.3801   -99.2279  19.1289   -99.0465
     13.0  22.1228   -97.7667  26.8429  -113.262 
      ⋮                                          
  99989.0  21.1633  -100.872   21.4098  -102.789 
  99990.0  19.9786   -99.1422  19.5902   -99.2825
  99991.0  19.3683   -98.9963  19.1912   -96.8645
  99992.0  16.1302   -94.0164  24.4229  -102.369 
  99993.0  26.1726  -112.232   24.1668   -99.9057
  99994.0  21.1808   -9

In [4]:
row, col = size(data)
print(row)

100000

In [5]:
using DataFrames
df = DataFrame()

""


In [16]:
for r in 1:100
    id = convert(Int64,data[r,1])
    lat = data[r,4]
    long = data[r,5]
    dict = reverse_geocoding(lat,long,api=:openstreetmap)
    #println(get(dict,"display_name","NA"),"NA")
    country = get(get(dict,"address","NA"),"country","NA")
    state = get(get(dict,"address","NA"),"state","NA")
    postcode = get(get(dict,"address","NA"),"postcode","NA")
    county = get(get(dict,"address","NA"),"county","NA")
    df[Symbol("$r")] = [id,lat,long,country,state,postcode,county]
end

In [8]:
data[1817,4:5]

2-element Array{Float64,1}:
  23.1536
 -97.8963

In [9]:
tmp_dict = reverse_geocoding(23.1536,-97.8963,api=:openstreetmap)
typeof(tmp_dict)

#Result
#{"place_id":"50676652","licence":"Data © OpenStreetMap contributors, ODbL 1.0. http:\/\/www.openstreetmap.org\/copyright","osm_type":"node","osm_id":"3738017964","lat":"22.9203035","lon":"-98.0899807","display_name":"Aldama, Tamaulipas, México","address":{"city":"Aldama","county":"Aldama","state":"Tamaulipas","country":"México","country_code":"mx"},"boundingbox":["22.6003035","23.2403035","-98.4099807","-97.7699807"]}


Dict{String,Any}

In [17]:
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
2,19.19059131,19.15473514,19.34936454,20.15710054,19.48320187,20.26814799,19.47968825,19.08838527,19.116567,16.41526089,19.20372485,19.12893426,26.84292035,19.63680661,20.43444983,19.33428921,19.72859362,21.2370931,18.87189484,21.15428364,21.35287969,19.13223568,19.01444338,28.29236937,17.25572775,20.66797288,18.63233565,19.3626295,17.07206383,19.11737609,19.38308301,18.4319,16.11497365,22.27846764,19.28463244,21.01213222,19.17695842,19.47684976,19.70264183,20.03210922,19.59388741,18.88974563,21.28473317,18.97873898,19.18070199,17.78126185,19.51288856,19.01764785,19.17024977,21.73214022,19.22226113,19.11247967,22.59788102,26.65836257,19.70954571,23.71991808,19.46839292,19.81883334,25.96507372,20.49093715,29.15685693,19.23351018,30.37635916,24.81382853,18.13091419,18.84822014,19.3851534,21.30390787,19.89751351,19.48443216,19.38393163,21.21173038,18.85298835,19.19985882,19.45084284,19.07274833,18.07123499,29.12590793,21.2768141,18.6612528,21.13867983,17.47260388,18.05267581,25.56915314,18.42654558,19.31001535,19.27676913,19.15895735,19.1922186,25.65713888,23.86867507,26.96654934,19.087521,18.79912326,19.29919809,19.83409726,18.75847784,17.78341295,27.15599139,19.03184634
3,-99.13981818,-99.02751399,-101.2246579,-100.1203004,-99.12241663,-99.90119526,-99.18284749,-99.01135515,-99.14083247,-96.64458885,-99.31002087,-99.04645417,-113.2620414,-99.24150119,-88.84542131,-97.62558085,-99.16582926,-99.6117513,-99.71150263,-101.4365262,-99.35839573,-99.17395135,-99.98826961,-109.4087386,-93.98593499,-101.6946138,-100.0829328,-100.3467352,-91.56984729,-99.64083191,-99.27351293,-97.87178831,-93.19614206,-104.423718,-99.19822416,-100.4148914,-98.98902562,-97.08449198,-99.98284588,-100.0624599,-99.2672167,-99.73607926,-100.6048926,-97.88266194,-99.13798401,-94.76911244,-99.19078411,-98.23211643,-99.05380151,-102.5416242,-98.96738482,-98.74278839,-103.3781656,-101.4384427,-98.28812469,-99.90420357,-98.35439673,-98.96494301,-99.89991361,-105.4342584,-101.4994208,-99.72564071,-115.0153979,-100.2717809,-97.79384902,-97.59975043,-99.09559604,-104.0880174,-101.0045412,-99.72618659,-99.04202562,-102.684894,-96.71965003,-99.20574612,-99.13510665,-99.08733263,-92.074237,-105.6807834,-99.96595716,-99.95531067,-104.0100738,-94.53325768,-101.7584247,-99.40208745,-95.58786844,-99.32954419,-99.34121261,-99.09438314,-98.9516928,-98.70433215,-105.9809854,-100.3211237,-100.5108179,-99.98064381,-99.20995518,-97.06741961,-99.79510197,-92.92149684,-100.1535911,-99.40816775
4,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México,México
5,CDMX,CDMX,Michoacán de Ocampo,Querétaro,CDMX,México,CDMX,NA,CDMX,Oaxaca,México,CDMX,Baja California Sur,México,Yucatán,Tlaxcala,México,Querétaro,México,Guanajuato,Querétaro,CDMX,México,Sonora,Veracruz de Ignacio de la Llave,Gu

In [18]:
df2 = convert(DataArray,df)

7×100 DataArrays.DataArray{Any,2}:
   1             2                 3                       …  100         
  19.1906       19.1547           19.3494                      19.0318    
 -99.1398      -99.0275         -101.225                      -99.4082    
    "México"      "México"          "México"                     "México" 
    "CDMX"        "CDMX"            "Michoacán de Ocampo"        "México" 
    "14500"       "NA"              "NA"                   …     "NA"     
    "Tlalpan"     "Milpa Alta"      "Morelia"                    "Ocuilan"

In [19]:
df3 = transpose(df2)

100×7 DataArrays.DataArray{Any,2}:
   1  19.1906   -99.1398  "México"  …  "Tlalpan"                    
   2  19.1547   -99.0275  "México"     "Milpa Alta"                 
   3  19.3494  -101.225   "México"     "Morelia"                    
   4  20.1571  -100.12    "México"     "Amealco de Bonfil"          
   5  19.4832   -99.1224  "México"     "Gustavo A. Madero"          
   6  20.2681   -99.9012  "México"  …  "Polotitlán"                 
   7  19.4797   -99.1828  "México"     "Azcapotzalco"               
   8  19.0884   -99.0114  "México"     "Cuauhtémoc"                 
   9  19.1166   -99.1408  "México"     "Tlalpan"                    
  10  16.4153   -96.6446  "México"     "Miahuatlán de Porfirio Díaz"
  11  19.2037   -99.31    "México"  …  "Xalatlaco"                  
  12  19.1289   -99.0465  "México"     "Milpa Alta"                 
  13  26.8429  -113.262   "México"     "Mulegé"                     
   ⋮                                ⋱                               